In [49]:
#!/usr/bin/env python3
"""lp_diff.py

Compara dos archivos LP/MPS y lista **solo** las familias de
restricciones que difieren entre ellos.

* Escala a millones de líneas usando un conjunto por archivo.
* Mantiene PEP 8, tipado y buen orden lógico.
* Reduce CPU evitando búsquedas innecesarias y usando expresiones
  regulares pre‑compiladas.

Autor:  María Cecilia Pérez — 2025‑06 (MIT)
"""
from __future__ import annotations

import re
import sys
from pathlib import Path
from typing import Dict, Iterable, Set
from collections import Counter

###############################################################################
#  CONSTANTES Y EXPRESIONES REGULARES                                       #
###############################################################################

_SECTION_KEYS: Dict[str, tuple[str, ...]] = {
    "objective": ("minimize", "maximize"),
    "constraints": ("subject to", "such that", "st", "s.t."),
    "bounds": ("bounds",),
    "generals": ("general", "generals"),
    "binary": ("binary",),
}

# Compilamos una sola vez para no recrear regex en cada línea
_SECTION_RE = {
    name: re.compile(rf"^\s*(?:{'|'.join(keys)})\b", re.I)
    for name, keys in _SECTION_KEYS.items()
}

_COMMENT_RE = re.compile(r"^\s*\\")
_CONSTR_START_RE = re.compile(r".+?:")               # línea con ‘:’
_REL_OP_RE = re.compile(r"(<=|>=|=)")                # operador relacional
_FAMILY_RE = re.compile(r"^([A-Za-z][A-Za-z0-9]*)(?:_|\{)")

###############################################################################
#  UTILIDADES DE AGRUPACIÓN POR FAMILIA                                     #
###############################################################################

def _family_key(line: str) -> str:
    """Devuelve la familia de la restricción (prefijo antes de ‘_’ o ‘{’)."""
    m = _FAMILY_RE.match(line)
    return m.group(1) if m else "sin_familia"


def _family_summary(lines: Iterable[str]) -> dict[str, int]:
    """Cuenta cuántas restricciones hay por familia."""
    summary: dict[str, int] = {}
    for ln in lines:
        fam = _family_key(ln)
        summary[fam] = summary.get(fam, 0) + 1
    return summary

###############################################################################
#  PARSER DE RESTRICCIONES                                                  #
###############################################################################

def _detect_section(line: str) -> str | None:
    """Retorna el nombre de sección o *None* si no inicia ninguna."""
    low = line.lower()
    for name, rex in _SECTION_RE.items():
        if rex.match(low):
            return name
    return None


def _normalize_constraint(raw: str) -> str:
    """Colapsa espacios y estandariza ‘:’ y operandos."""
    raw = re.sub(r"\s+", " ", raw.strip())
    raw = re.sub(r"\s*:\s*", ": ", raw)
    raw = re.sub(r"\s*(<=|>=|=)\s*", r" \1 ", raw)
    return raw


def _constraints(path: Path):
    """Devuelve: (set_restricciones, total_líneas, Counter_por_sección)."""
    cons: Set[str] = set()
    in_constraints = False
    buf: list[str] = []

    total_lines = 0
    sec_counter: Counter[str] = Counter()
    current_sec: str | None = None

    with path.open(encoding="utf-8", errors="ignore") as fh:
        for raw in fh:
            total_lines += 1
            line = raw.strip()

            # ­— líneas vacías o comentarios —
            if not line or _COMMENT_RE.match(line):
                continue

            # ­— ¿empieza sección? —
            if (sec := _detect_section(line)) is not None:
                current_sec = sec
                sec_counter[sec] += 1

                # guardar lo que llevábamos si estábamos dentro de Constraints
                if in_constraints and buf:
                    cons.add(_normalize_constraint(" ".join(buf)))
                    buf.clear()

                in_constraints = sec == "constraints"
                continue

            # ­— fin absoluto —
            if line.lower().startswith("end"):
                break

            # ­— líneas fuera de constraints: solo sumamos estadística —
            if not in_constraints:
                if current_sec:
                    sec_counter[current_sec] += 1
                continue

            # ­— dentro de Constraints —
            if _CONSTR_START_RE.match(line) and buf:
                cons.add(_normalize_constraint(" ".join(buf)))
                buf.clear()

            buf.append(line)

            if _REL_OP_RE.search(line):
                cons.add(_normalize_constraint(" ".join(buf)))
                buf.clear()

    # último bloque pendiente
    if buf:
        cons.add(_normalize_constraint(" ".join(buf)))

    return cons, total_lines, sec_counter


###############################################################################
#  DIF Y REPORTE                                                            #
###############################################################################

def _diff(a: Set[str], b: Set[str]) -> tuple[Set[str], Set[str]]:
    """Diferencias simples: (solo en *a*, solo en *b*)."""
    return a - b, b - a


def _print_report(name_a: str, name_b: str,
                  only_a: Set[str], only_b: Set[str]) -> None:
    """Imprime únicamente las familias de restricciones distintas."""
    if not only_a and not only_b:
        print("✅  Los archivos tienen las mismas familias de restricciones.")
        return

    print("🔍  Familias de restricciones que difieren\n")

    if only_a:
        fams_a = _family_summary(only_a)
        left = " | ".join(f"{k} ({v})" for k, v in sorted(fams_a.items()))
        print(f"• Solo en {name_a}: {left}")

    if only_b:
        fams_b = _family_summary(only_b)
        right = " | ".join(f"{k} ({v})" for k, v in sorted(fams_b.items()))
        print(f"• Solo en {name_b}: {right}")

def _print_comparison_table(nom_a: str,
                            lines_a: int, fams_a: int, secs_a: Counter,
                            nom_b: str,
                            lines_b: int, fams_b: int, secs_b: Counter) -> None:
    """
    Muestra lado-a-lado el resumen de ambos modelos
    (líneas, familias y recuento por sección).
    """
    # 1) Prepara filas ----------------------------------------------------------------
    filas = [
        ("Líneas analizadas",        lines_a, lines_b),
        ("Familias de restricciones",fams_a,  fams_b),
    ]

    # Unión de claves de sección (p. ej. objective, constraints…)
    orden_pref = ("objective", "constraints", "bounds", "generals")
    todas = sorted({*secs_a.keys(), *secs_b.keys()},
                   key=lambda k: orden_pref.index(k) if k in orden_pref else 99)

    for k in todas:
        filas.append((k.capitalize(), secs_a.get(k, 0), secs_b.get(k, 0)))

    # 2) Cálculo de anchos -------------------------------------------------------------
    w0 = max(len(f[0]) for f in filas)
    w1 = max(len(f"{f[1]:,}") for f in filas)
    w2 = max(len(f"{f[2]:,}") for f in filas)

    # 3) Cabecera ----------------------------------------------------------------------
    print("\n📊  Comparativo de métricas")
    linea = f"{'Sección':{w0}} | {nom_a:>{w1}} | {nom_b:>{w2}}"
    print(linea)
    print("-" * len(linea))

    # 4) Filas -------------------------------------------------------------------------
    for lab, v1, v2 in filas:
        print(f"{lab:{w0}} | {v1:>{w1},} | {v2:>{w2},}")
    print()  # línea en blanco de cortesía


In [80]:
###############################################################################
#  MAIN                                                                     #
###############################################################################

"""Punto de entrada: lee dos rutas y muestra el resumen."""
if len(sys.argv) == 3:
    lp_a, lp_b = map(Path, sys.argv[1:])
else:
    # 👉 Hard‑coded para uso rápido; modifica a tu gusto
    lp_a = Path(
        r"E:\Entrenamiento\20250317_Look_ahead_pruebas\PID_20250219_01\PID_20250219"
        r"\Modelos\Model Test15d Solution R08 sin look ahead\Model ( Test15d ) ST n=1 s=1 i=0.lp"
    )
    lp_b = Path(
        r"E:\Entrenamiento\20250317_Look_ahead_pruebas\PID_20250219_01\PID_20250219"
        r"\Modelos\Model Test15d Solution_always\Model ( Test15d ) ST n=1 s=1 i=0.lp"
    )

if not lp_a.is_file() or not lp_b.is_file():
    sys.exit("❌ Alguno de los archivos .lp no existe o no es legible.")

cons_a, lines_a, secs_a = _constraints(lp_a)
cons_b, lines_b, secs_b = _constraints(lp_b)

# NUEVO: obtener diff antes de imprimir
only_a, only_b = _diff(cons_a, cons_b)

_print_comparison_table(
    lp_a.name, lines_a, len(cons_a), secs_a,
    lp_b.name, lines_b, len(cons_b), secs_b
)

_print_report(lp_a.name, lp_b.name, only_a, only_b)




📊  Comparativo de métricas
Sección                   | Model ( Test15d ) ST n=1 s=1 i=0.lp | Model ( Test15d ) ST n=1 s=1 i=0.lp
-----------------------------------------------------------------------------------------------------
Líneas analizadas         | 4,070,434 | 8,090,547
Familias de restricciones | 1,075,487 | 2,149,631
Objective                 |   115,252 |   165,729
Constraints               |         1 |         1
Bounds                    |   214,590 |   429,118
Generals                  |    42,038 |    84,386

🔍  Familias de restricciones que difieren

• Solo en Model ( Test15d ) ST n=1 s=1 i=0.lp: BatRecycle (1) | Con (167) | ConTotCarry (39) | GenMaxStartsWeek (5) | StoRecycle (16) | StoValVolDef (7)
• Solo en Model ( Test15d ) ST n=1 s=1 i=0.lp: BatBal (48) | BatMaxCycDay (2) | BatMaxCycDayActDef (48) | BatPmax (48) | BatRecycle (1) | BatRelDef (48) | Con (16261) | ConActDef (8208) | ConTotCarry (39) | FueMaxOffDay (8) | FueMaxOffDayActDef (192) | GenCommit (288) | 

In [83]:
#Model Test15d Solution R08 sin look ahead 4 dias
con = sorted(c for c in only_a if c.startswith("BatRecycle"))
con

['BatRecycle_Head_BAT_ARICA: BatEndSoC_Head_BAT_ARICA{48} = 0']

In [84]:
#Model Test15d Solution R08 con look ahead condiciones normales 1 hora y ajuste G
con = sorted(c for c in only_b if c.startswith("BatRecycle"))
con

['BatRecycle_Head_BAT_ARICA: BatEndSoC_Head_BAT_ARICA{96} = 0']